# 라이브러리

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# matplotlib 그래프 속성 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = (12,16)
plt.rcParams['font.size'] = 20

# 그래프 그릴 때 마이너스(-) 부분도 표시해주기 (로그오즈비 표현을 위해 세팅)
plt.rcParams['axes.unicode_minus'] = False

# 사용자 정의 함수

In [11]:
# 텍스트 전처리
def prepro_text(raw_data):

    # 특수기호, 영어, 숫자 제거
    prepro_texts = re.sub(r'[^가-힣]',' ',str(raw_data))

    # 형태소 분석기 생성
    okt = Okt()

    # 조사와 복수표현 등 필요없는 품사 tag 제거
    prepro_word = []
    for word, tag in okt.pos(prepro_texts):
        if tag not in ['Josa', 'Suffix']:
            prepro_word.append(word)
    
    # 어간 추출 적용
    # result = ' '.join(okt.morphs(' '.join(prepro_word), stem=True))

    result = ' '.join(prepro_word)

    return result


# 빈도분석 후 결과 출력
def count_analyze(texts, count_vec, color=None, title=None):
    
    count_vec.fit(texts)
    
    word_dict = sorted(count_vec.vocabulary_.items())
    idx2word = {idx:word for word, idx in word_dict}

    total_text = []
    total_text.append(' '.join(texts.values))

    count_matrix = count_vec.transform(total_text)

    count_word = []
    count_vector = []

    for i in range(20,0,-1):
        count_word.append(idx2word[(-count_matrix.toarray()[0]).argsort()[i-1]])
        count_vector.append(count_matrix.toarray()[0][(-count_matrix.toarray()[0]).argsort()[i-1]])

    # print(count_word)
    # print(count_vector)

    # plt.barh(count_word, count_vector, color=color)
    # plt.yticks(count_word)
    # plt.title(f'{title} 빈도 분석')
    # plt.show()

    return count_word, count_vector


# 코사인 유사도 score 추출
def cosine_extraction(vec, fit_data, count_words):

    # 적합 후 벡터 변화
    analysis_vec = vec.fit_transform(fit_data)    

    # 빈도분석으로 뽑은 Top10 단어들 하나의 text로
    top10_word_vec = vec.transform([' '.join(count_words)])

    cosine_sim = cosine_similarity(top10_word_vec, analysis_vec)

    cos_sim = fit_data.name

    cosine_df = pd.DataFrame({
        'h_id' : fit_data.index.values,
        cos_sim : cosine_sim[0]
    })
    #fit_cosine_df['코사인 유사도'] = fit_cosine_df['코사인 유사도'].map(lambda x: round(x,3))
    #fit_cosine_df['rank'] = fit_cosine_df['코사인 유사도'].rank(ascending=False).astype(np.int64)

    return cosine_df

# 데이터 로드

In [14]:
pd.set_option('display.max_row', 6)

In [ ]:
path = 'C:\\Users\\GOOD\\Desktop\\이승훈\\hjgrace\\신사업프로젝트\\히든그레이스_자소서_텍스트분석_data.xlsx'
df = pd.read_excel(path)
df

In [8]:
# 내가 필요한 데이터 컬럼 index 확인
for idx, title in enumerate(df.columns.values):
    if title == 'label2':
        print(idx, title)
    elif title == '히든그레이스는 어떤 회사인가':
        print(idx, title)
    elif title == '히든그레이스가 보완할 점':
        print(idx, title)
    elif title == '채용 된다면 일할 수 있는 기간, 오래 일하는 데 문제 되는 요소':
        print(idx, title)

17 label2
25 히든그레이스는 어떤 회사인가
26 히든그레이스가 보완할 점
27 채용 된다면 일할 수 있는 기간, 오래 일하는 데 문제 되는 요소


# 데이터 전처리

In [ ]:
# 전체 데이터
use_df = df.iloc[:,[0,17,25,26,27]]

for col in use_df.columns.values[-3:]:
    use_df[col] = use_df[col].map(prepro_text)

### 데이터 분리

In [6]:
# label0인 데이터 (새로운 데이터가 포함된)
label0_df = use_df[use_df.iloc[:,1]==0]

# label1인 데이터 (새로운 데이터가 포함된)
label1_df = use_df[use_df.iloc[:,1]==1]
label1_df = pd.concat([label1_df, use_df[-3:]], axis=0)


# 코사인 유사도 추출

## label0

In [ ]:
# 코사인 유사도 넣을 dataframe 생성
label0_cos_df = pd.DataFrame({
    'h_id' : label0_df.index.values,
    'name' : label0_df.iloc[:,0].values,
    })

# 코사인 유사도 추출 & merge
for col in label0_df.columns[-3:]:
    
    # fit할 데이터
    fit_data = label0_df[col]
    # 불용어사전
    stop_words = ['하는', '있는', '하여', '합니다', '하게', '지원', '입니다', '입사', '하겠습니다', '위해', '되었습니다', '싶습니다', '가지', '생각', '통해',
                  '회사', '히든', '그레이스', '분야', '해야', '같습니다', '관련', '오래', '하는데', '요소', '되는', '된다면', '있습니다', '하지', '정도', '하고',
                  '있을', '만약', '어떤', '때문', '있다고']
    # count 벡터라이저
    count_vec = CountVectorizer(
        stop_words=stop_words,
        min_df=2
    )
    # 빈도분석 결과
    label0_word, label0_vec = count_analyze(fit_data, count_vec)
    # 코사인 유사도 
    cos_df = cosine_extraction(count_vec, fit_data, label0_word)
    # merge
    label0_cos_df = pd.merge(label0_cos_df, cos_df, on='h_id', how='left')

# 전체 평균 구하기
label0_cos_df['mean'] = label0_cos_df.iloc[:,2:].mean(axis=1)
# 랭킹 구하기
label0_cos_df['rank'] = label0_cos_df['mean'].rank(ascending=False).astype(np.int64)

label0_cos_df.sort_values(by='rank', axis=0, ascending=True, inplace=True)
label0_cos_df


## label1

In [ ]:
# 코사인 유사도 넣을 dataframe 생성
label1_cos_df = pd.DataFrame({
    'h_id' : label1_df.index.values,
    'name' : label1_df.iloc[:,0].values,
    })

# 코사인 유사도 추출 & merge
for col in label1_df.columns[-3:]:
    
    # fit할 데이터
    fit_data = label1_df[col]
    # 불용어사전
    stop_words = ['하는', '있는', '하여', '합니다', '하게', '지원', '입니다', '입사', '하겠습니다', '위해', '되었습니다', '싶습니다', '가지', '생각', '통해',
                  '회사', '히든', '그레이스', '분야', '해야', '같습니다', '관련', '오래', '하는데', '요소', '되는', '된다면', '있습니다', '하지', '정도', '하고',
                  '있을', '만약', '어떤', '때문', '있다고']
    # count 벡터라이저
    count_vec = CountVectorizer(
        stop_words=stop_words,
        min_df=2
    )
    # 빈도분석 결과
    label0_word, label0_vec = count_analyze(fit_data, count_vec)
    # 코사인 유사도 
    cos_df = cosine_extraction(count_vec, fit_data, label0_word)
    # merge
    label1_cos_df = pd.merge(label1_cos_df, cos_df, on='h_id', how='left')

# 전체 평균 구하기
label1_cos_df['mean'] = label1_cos_df.iloc[:,2:].mean(axis=1)
# 랭킹 구하기
label1_cos_df['rank'] = label1_cos_df['mean'].rank(ascending=False).astype(np.int64)

label1_cos_df.sort_values(by='rank', axis=0, ascending=True, inplace=True)
label1_cos_df
